In [1]:
%pip install langdetect nltk sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 578.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 2.2 MB/s eta 0:00:00 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.2 which is incompatible.


# Движок полнотекстового поиска

In [2]:
%pip install natasha

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from langdetect import detect
from concurrent.futures import ThreadPoolExecutor
from nltk import SnowballStemmer
from tqdm import tqdm
from gensim.models import Word2Vec


class FullText:
    def __init__(self, data):
        self.idx = self.index_data(data)
        

    def get_stem(self, sentence: str) -> list[str]:
        language = 'russian'
        try:
            language = 'english' if detect(sentence) != 'ru' else 'russian'
        except:
            pass

        stemmer = SnowballStemmer(language)
        my_words = sentence.split()
        return [stemmer.stem(word) for word in my_words]

    def index_data(self, data):
        data_idx = {}
        
        for i in tqdm(range(0, 7000000)):
            try:
                title = str(data.loc[i, 'video_title'])
                video_id = data.loc[i, 'video_id']

                stems = self.get_stem(title)
                for stem in stems:
                    if stem in data_idx:
                        data_idx[stem].append(video_id)
                    else:
                        data_idx[stem] = [video_id]
                            
            except TypeError:
                print("Huy")
                    
        return data_idx

    @staticmethod
    def transliterate(name):
        # Словарь с заменами
        slovar_1 = {'ай': 'i', 'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'дж': "j", 'д': 'd', 'ей': 'ay', 'е': 'e',
                    'ё': 'yo',
                    'ж': 'j', 'з': 'z', 'и': 'i', 'й': 'i', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n',
                    'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'h',
                    'ц': 'c', 'ч': 'ch', 'ш': 'sh', 'щ': 'sch', 'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e',
                    'ю': 'u', 'я': 'ya'}

        slovar_2 = {'ай': 'i', 'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'дж': "j", 'д': 'd', 'ей': 'ay', 'е': 'e',
                    'ё': 'yo',
                    'ж': 'j', 'з': 'z', 'и': 'i', 'й': 'i', 'к': 'c', 'л': 'l', 'м': 'm', 'н': 'n',
                    'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'h',
                    'ц': 'c', 'ч': 'ch', 'ш': 'sh', 'щ': 'sch', 'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e',
                    'ю': 'u', 'я': 'ya'}

        name = name.lower()

        translit_1 = name
        for key in slovar_1:
            translit_1 = translit_1.replace(key, slovar_1[key])

        translit_2 = name
        for key in slovar_2:
            translit_2 = translit_2.replace(key, slovar_2[key])

        return [translit_1, translit_2]

    def search(self, query):
        stems = self.get_stem(query)
        result = []
        for stem in stems:
            if stem in self.idx:
                result += self.idx[stem]

        for translit in self.transliterate(query):
            stems_t = self.get_stem(translit)
            for stem in stems_t:
                if stem in self.idx:
                    result += self.idx[stem]

        return set(result)


# Движок семантического поиска

In [4]:
import sentence_transformers
from sentence_transformers import SentenceTransformer


class Semantic:
    def __init__(self, sentence_model: SentenceTransformer, threshold=0.5):
        self.model = sentence_model
        self.threshold = threshold

    def get_semantic_list(self, query: str, data: list[str]):
        semantic_result = []

        query_sentence = self.model.encode(query)
        embeddings = self.model.encode(data)

        i = 0

        for sentence in embeddings:
            s = sentence_transformers.util.cos_sim(a=sentence, b=query_sentence)
            if s > self.threshold:
                semantic_result.append(data[i])

        return semantic_result


# Парсинг данных

In [5]:
import pandas as pd

data = pd.read_parquet(
    "videos.parquet",
    engine="fastparquet",
    columns=[
        "video_id",
        "video_title",
    ],
)

In [6]:
len(data.axes[0])

34404561

In [7]:
# Пример использования
import sentence_transformers

def find_by_key(iterable, key, value):
    for index, dict_ in enumerate(iterable):
        if key in dict_ and dict_[key] == value:
            return dict_


fulltext = FullText(data)

model = sentence_transformers.SentenceTransformer('inkoziev/sbert_pq' )
semantic = Semantic(model, 0.2)

while True:
    query = input("Query: ")
    response = []

    results = fulltext.search(query)
    # поиск названия по id видео
    for res in results:
        response.append(find_by_key(data, "video_id", res)['video_title'])

    results = semantic.get_semantic_list(query, response)
    print(results)



  1%|          | 35405/7000000 [03:25<11:14:19, 172.14it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-2178c6e9b86b>", line 10, in <module>
    fulltext = FullText(data)
  File "<ipython-input-3-b48e594a95c7>", line 10, in __init__
    self.idx = self.index_data(data)
  File "<ipython-input-3-b48e594a95c7>", line 30, in index_data
    video_id = data.loc[i, 'video_id']
  File "/home/jupyter/.local/lib/python3.10/site-packages/pandas/core/indexing.py", line 1145, in __getitem__
    if self._is_scalar_access(key):
  File "/home/jupyter/.local/lib/python3.10/site-packages/pandas/core/indexing.py", line 1223, in _is_scalar_access
    if len(key) != self.ndim:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2045, in showt

TypeError: object of type 'NoneType' has no len()

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загрузка данных
df = pd.read_parquet(
    "features.parquet",
    engine="fastparquet",
    columns=[
        "video_id",
        "v_category",
        "v_pub_datetime",
        "v_likes",
        "v_dislikes",
        "v_comments",
        "v_cr_click_long_view_7_days"
    ],
)

# Функция для определения весов в зависимости от категории видео
def get_weights(category):
    if category in ["Телепередачи", "Спорт", "Спорт/Игры"]:
        return {"v_pub_datetime": 0.4, "v_dislikes": 0.25, "v_likes": 0.2,  "v_cr_click_long_view_7_days": 0.1, "v_comments": 0.1}
    else:
        return {"v_dislikes": 0.25, "v_likes": 0.3, "v_cr_click_long_view_7_days": 0.2, "v_comments": 0.1, "v_pub_datetime": 0.1}

# Добавление столбца с категорией видео
df["category"] = df["v_category"]  # Замените "category_column_name" на имя вашего столбца с категорией

# Определение весов для каждого фактора (важности)
df["weights"] = df["category"].apply(get_weights)

# Нормализация данных
scaler = StandardScaler()
df[["v_pub_datetime", "v_likes", "v_dislikes", "v_cr_click_long_view_7_days", "v_comments"]] = scaler.fit_transform(
    df[["v_pub_datetime", "v_likes", "v_dislikes", "v_cr_click_long_view_7_days", "v_comments"]]
)

# Преобразование времени публикации в числовой формат
df["v_pub_datetime"] = pd.to_datetime(df["v_pub_datetime"]).astype(int)

# Разделение данных на обучающую и тестовую выборки
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Создание Pool для обучения
train_pool = Pool(
    train_data[["v_cr_click_long_view_7_days", "v_likes", "v_dislikes", "v_comments", "v_pub_datetime"]],
    label=train_data["label"],
    cat_features=["categorical_feature"],
)

# Инициализация модели CatBoost
model = CatBoostRegressor(
    iterations=1000, depth=6, learning_rate=0.1, loss_function="RMSE"
)

# Обучение модели
model.fit(train_pool, verbose=100)

# Использование модели для ранжирования тестовых данных
test_pool = Pool(
    test_data[["v_cr_click_long_view_7_days", "v_likes", "v_dislikes", "v_comments", "v_pub_datetime"]],
    cat_features=["categorical_feature"],
)
test_data["predicted_rank"] = model.predict(test_pool)

# Сортировка тестовых данных по предсказанному рангу
test_data = test_data.sort_values(by="predicted_rank", ascending=False)
print(test_data[["video_id", "predicted_rank"]])
